In [16]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [17]:
events = utils.get_events()

### Feature de TFIdf

In [18]:
# Comenzaremos realizando un TF_idf de as busquedas
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [19]:
# Me quedo con los usuarios que tiene busquedas
busquedas = events[events['search_term'].notnull()][['search_term','person']]

# Calculo de Tf_Idf
busquedas_totales = busquedas['search_term'].tolist()
IDF = vectorizer.fit_transform(busquedas_totales)

# Asigno los TFIdf a cada busqueda
resultados = []
for x in range(len(busquedas)):
    resultados.append(IDF[x].sum()) 

busquedas['TfIdf'] = resultados
busquedas_tfidf = busquedas[['person','TfIdf']]

# Agrupo por persona y calculo el promedio de los TFIdf
busquedas_por_usuario = busquedas_tfidf.groupby('person')['TfIdf'].agg({'promedio de TFIdf':'mean'}).reset_index()

In [20]:
busquedas_por_usuario.fillna(0,inplace=True)
busquedas_por_usuario.head()

,person,promedio de TFIdf
0,000c79fe,1.168325
1,000e619d,1.387051
2,001001be,1.000000
3,001802e4,1.277478
4,0019e639,1.163643


# Feature len promedio de las busquedas


In [21]:
busquedas['longitud busqueda'] = busquedas['search_term'].apply(lambda x: len(str(x)))
busquedas.drop(columns='search_term',inplace=True)
busquedas.head()

,person,TfIdf,longitud busqueda
5,4c8a8b93,1.303777,9
18,29ebb414,1.000000,6
29,15ea8012,1.715170,14
35,d21b8e6b,1.355329,7
39,d21b8e6b,1.388455,7


In [22]:
# Ahora calculo el promedio de las busquedas por usuario
len_busquedas_por_usuario = busquedas.groupby('person')['longitud busqueda'].mean().to_frame().reset_index()
len_busquedas_por_usuario.head()

,person,longitud busqueda
0,000c79fe,8.500000
1,000e619d,10.333333
2,001001be,7.250000
3,001802e4,10.000000
4,0019e639,7.090909


In [23]:
busquedas_por_usuario = utils.add_features(len_busquedas_por_usuario,busquedas_por_usuario,'person','left')
busquedas_por_usuario.fillna(0,inplace=True)
busquedas_por_usuario.sort_values(by='person', inplace=True)
busquedas_por_usuario.head()

,person,longitud busqueda,promedio de TFIdf
0,000c79fe,8.500000,1.168325
1,000e619d,10.333333,1.387051
2,001001be,7.250000,1.000000
3,001802e4,10.000000,1.277478
4,0019e639,7.090909,1.163643


In [6]:
busquedas_por_usuario.to_csv('features_busquedas.csv',index=False)